In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_radars = pd.read_csv("radars.csv")
df_radars

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(df_radars, lat="latitude", lon="longitude", hover_name="emplacement", color_discrete_sequence=["blue"], zoom=5, height=850, width=850)

fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [ ]:
temp = df_radars.groupby(["emplacement"])
radars_dep = temp[["id"]].count()
radars_dep['latitude'] = temp[['latitude']].mean()
radars_dep['longitude'] = temp[['longitude']].mean()
radars_dep

In [ ]:
dep = px.scatter_mapbox(radars_dep,
                        lat="latitude",
                        lon="longitude",
                        color_discrete_sequence=["blue"],
                        zoom=5, height=850, width=850,
                        size="id", color="id", hover_name=radars_dep.index,
                        color_continuous_scale=px.colors.carto.Purpor_r)

dep.update_layout(mapbox_style="open-street-map")
dep.show()

In [ ]:
radars_year = df_radars[["date_installation", "emplacement", "latitude", "longitude", "id"]]
radars_year["date_installation"] = pd.DatetimeIndex(radars_year['date_installation']).year
radars_year

In [ ]:
from dash import Dash, dcc, html, Input, Output

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Interactive scatter plot with radars dataset'),
    dcc.Graph(id="scatter-mapbox"),
    html.P("Filter by year:"),
    dcc.Slider(2003, 2018, 1,
               value=2003,
               id="range-slider"
    ),
])


@app.callback(
    Output("scatter-mapbox", "figure"), 
    Input("range-slider", "value"))
def update_bar_chart(slider_range):
    df = radars_year # replace with your own data source
    value = slider_range
    mask = df['date_instalaltion'] <= value
    fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name="emplacement", color_discrete_sequence=["blue"], zoom=5, height=850, width=850)
    return fig


app.run_server(debug=True)